In [1]:
def get_file_name():
    return input("Enter the input file name (with extension, e.g., file.txt): ").strip()


In [2]:

import tkinter as tk
from tkinter import messagebox

def warningbox():
    """
    Display a warning message box to notify the user and stop execution.
    """
    root = tk.Tk()
    root.withdraw()  # Hide the main tkinter window
    messagebox.showinfo("Attention", "Folder created. Put your input file in the new folder.")
    root.destroy()
    #exit  # Stop execution

In [3]:

from pathlib import Path

# fonction pour créer un nouveau dossier 
def create_folder(base_path, folder_name):
    folder_path = base_path / folder_name
    if not folder_path.exists():
        folder_path.mkdir(parents=True)
        print(f"Folder created: {folder_path}")
        warningbox()  # Show warning and stop execution
    else:
        print(f"Folder already exists: {folder_path}")
    return folder_path



In [4]:
# fonction pour changer le nom d'un dossier existant
def rename_folder(base_path, old_name, new_name):
    old_path = Path(base_path) / old_name
    new_path = Path(base_path) / new_name
    
    if old_path.exists():
        old_path.rename(new_path)
        print(f"Folder renamed from {old_name} to {new_name}")
        return new_path
    else:
        print(f"Folder '{old_name}' does not exist.")
        return None

In [5]:
from xmindparser import xmind_to_dict
from pathlib import Path
import pprint

def main():
    home_dir = Path.home()  # ton home directory C/User/ton NNI
    subdir_path = home_dir / "OneDrive - EDF" / "Data_input_for_studies" / "xmind_tool"
    subdir_path.mkdir(parents=True, exist_ok=True)
    print(f"Using subdirectory path: {subdir_path}")

    # spécifier ou changer le nom du dossier
    action = input("Do you want to create or rename a folder? (create/rename/none): ").strip().lower()
    if action == "create":
        folder_name = input("Enter the name of the new folder: ").strip()
        folder_path = create_folder(subdir_path, folder_name)
    elif action == "rename":
        old_name = input("Enter the current folder name: ").strip()
        new_name = input("Enter the new folder name: ").strip()
        folder_path = rename_folder(subdir_path, old_name, new_name)
    else:
        folder_name = "DefaultFolder"
        folder_path = subdir_path / folder_name
        print(f"Using default folder path: {folder_path}")
        folder_path.mkdir(parents=True, exist_ok=True)

    # Donner un nom à ton input fichier (xmind)
    input_file_name = get_file_name()
    input_file_path = folder_path / input_file_name

    print(f"Constructed file path: {input_file_path}")

    # Optionally create the file if it doesn't exist
    if not input_file_path.exists():
        with open(input_file_path, 'w') as file:
            file.write("")  # Create an empty file
        print(f"File created: {input_file_path}")
    else:
        print(f"File already exists: {input_file_path}")
    
    # parsed_data = xmind_to_dict(input_file_path)
    return input_file_path


if __name__ == "__main__":
    input_file_path = main()
    print(input_file_path)

Using subdirectory path: C:\Users\jl0331el\OneDrive - EDF\Data_input_for_studies\xmind_tool


Folder renamed from topla to top
Constructed file path: C:\Users\jl0331el\OneDrive - EDF\Data_input_for_studies\xmind_tool\top\SYNOPTIQUES Martigues5 Test_Raja.xmind
File already exists: C:\Users\jl0331el\OneDrive - EDF\Data_input_for_studies\xmind_tool\top\SYNOPTIQUES Martigues5 Test_Raja.xmind
C:\Users\jl0331el\OneDrive - EDF\Data_input_for_studies\xmind_tool\top\SYNOPTIQUES Martigues5 Test_Raja.xmind


In [6]:
# Entrer ou modifier les noms des fichier

# nom du fichier input en xmind

# input_file_name = "MYPLANTONLINE CETAC.xmind"

# nom du fichier output en excel
output_file_name = "exported_data.xlsx"


In [8]:
from xmindparser import xmind_to_dict
from pathlib import Path


home_dir = Path.home()
# input_file = home_dir.joinpath("OneDrive - EDF", "XMind", input_file_name)
output_file = home_dir.joinpath("OneDrive - EDF", "XMind", "output_excel", output_file_name)
parsed_data = xmind_to_dict(input_file_path)
pprint.pprint(parsed_data)

[{'id': '5dl370qdfjmenl5a2cu0cak3ge',
  'structure': 'org.xmind.ui.logic.right',
  'title': 'Feuille  1',
  'topic': {'comment': None,
            'id': '0e60609e0gjjg54ee5v2ga6d85',
            'labels': None,
            'link': 'https://prod-thaf-vision.edf.fr/PIVision/#/Displays/1685',
            'makers': None,
            'note': None,
            'title': 'MYPLANTONLINE MARTIGUES 5 -SYNOPTIQUES ',
            'topics': [{'comment': None,
                        'id': '1ps7oh3jmgmqgr9n54jk83g5q9',
                        'labels': None,
                        'link': 'https://prod-thaf-vision.edf.fr/PIVision/#/Displays/1754',
                        'makers': None,
                        'note': None,
                        'title': 'prévision de fonctionnement'},
                       {'comment': None,
                        'id': '6en8o7lf4j6mt2j708th2kh2fi',
                        'labels': None,
                        'link': 'https://prod-thaf-vision.edf.fr/PIVision/

In [11]:
import pandas as pd

df = parsed_data

def extract_hierarchy(topics, path=None):
    """
    Faire une extraction de manière hierarchique et récursive
    de données dans les topics sousjacents.
    
    Args:
        topics (liste): Liste de topics.
        path (liste): Liste pour maintenir le trajectoir hierarchique.
    
    Returns:
        une liste de lignes où chaque ligne d'excel contient les titres hierarchique
        et un link.

    """
    if path is None:
        path = [] # initialiser la liste

    rows = []
    for topic in topics:
        current_path = path + [topic.get('title', '')]  # update la liste de path avec le titre actuel
        # vérifier si l'hyperlink exists au niveau actuel (la ligne actuelle)
        if 'link' in topic:
            rows.append(current_path + [topic['link']])
        else:
            # ajouter une ligne dans l'excel avec la valeur nulle d'hyperlink si l'hyperlink n'exist pas dans le dict
            rows.append(current_path + [''])
        
        # Récursion dans les subtopics
        if 'topics' in topic:
            rows.extend(extract_hierarchy(topic['topics'], current_path))
    return rows

# Création du root topic de la donnée parsé
root_topic = [{'title': df[0]['topic']['title'], 'link': df[0]['topic'].get('link'), 'topics': df[0]['topic']['topics']}]

# Extraction des lignes dans l'hierachie du dataframe
rows = extract_hierarchy(root_topic)

# Déterminer le longeur (profondeur) du titre => (titre level)
max_depth = max(len(row) - 1 for row in rows)

# Normalize rows to have equal columns (titles and link)
normalized_rows = [row[:-1] + [''] * (max_depth - len(row) + 1) + [row[-1]] for row in rows]

# Create dynamic column names for titles based on the maximum depth
columns = [f'Title Level {i+1}' for i in range(max_depth)] + ['Link']
df = pd.DataFrame(normalized_rows, columns=columns)

# Export to Excel
excel_file_path = output_file
df.to_excel(excel_file_path, index=False)

print(f"Excel file saved to: {excel_file_path}")


Excel file saved to: C:\Users\jl0331el\OneDrive - EDF\XMind\output_excel\exported_data.xlsx
